In [18]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, auc, roc_curve, precision_recall_curve
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [2]:
df = pd.read_csv('train.csv')
df.columns = df.columns.str.lower()
# df = df[[  'age', 'driving_license', 'region_code',
#        'vintage', "response"]]
train = df.iloc[:250000].copy()
test = df.iloc[250000:].copy()
Xtrain, ytrain = train.drop(columns="response"), train[["response"]]
Xtest, ytest = test.drop(columns="response"), test[["response"]]
train.head()

,age,driving_license,region_code,vintage,response
0,44,1,28.0,217,1
1,76,1,3.0,183,0
2,47,1,28.0,27,1
3,21,1,11.0,203,0
4,29,1,41.0,39,0


In [6]:
train["response"].value_counts(normalize=True)

0    0.877232
1    0.122768
Name: response, dtype: float64

In [33]:


def train_and_evaluate_classification_model(df, target_column, test_size_ratio, n_iter=5):
    """
    Train a classification model and evaluate various metrics.
    
    :param df: DataFrame containing the features and target.
    :param target_column: The name of the target variable column.
    :param test_size_ratio: Ratio for train_test_split.
    :return: Dictionary containing various evaluation metrics.
    """
    # Splitting the data
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_ratio, random_state=42)

     # Define the parameter grid
    param_dist = {
        'depth': [4, 6, 8, 10],
        'learning_rate': np.linspace(0.01, 0.2, 20),
        'iterations': [100, 200, 300, 500],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        'border_count': [32, 64, 128, 256],
        'bagging_temperature': [0, 1, 2, 3, 4],
        "scale_pos_weight": [4],
        "eval_metric":["prauc"]
    }

    # Initialize the CatBoostClassifier
    model = CatBoostClassifier(verbose=False,
                               one_hot_max_size=5,
                               # auto_class_weights='Balanced'
                                )
        

    # Initialize RandomizedSearchCV
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, 
                                       n_iter=n_iter, cv=3, n_jobs=-1, verbose=2, random_state=42)

    # Perform random search
    random_search.fit(X_train, y_train)

    # Return the best estimator
    model = random_search.best_estimator_

    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    precision_recall, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    aucpr = auc(recall, precision_recall)

    # Kolmogorov-Smirnov statistic
    ks_statistic = stats.ks_2samp(y_test, y_pred).statistic

    return {
        'roc_auc': roc_auc,
        'aucpr': aucpr,
        'ks_statistic': ks_statistic
    }

# Example usage:
# Ensure you have the required libraries installed: pandas, sklearn, catboost, scipy
# df = pd.read_csv('your_data.csv')  # Load your dataset
# results = train_and_evaluate_classification_model(df, 'target_column_name', 0.2)  # Replace with your target column name
# print(results)


In [34]:
train_and_evaluate_classification_model(df, "response",0.7)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


ValueError: 
All the 15 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\kosmo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\kosmo\AppData\Local\Programs\Python\Python310\lib\site-packages\catboost\core.py", line 5100, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "c:\Users\kosmo\AppData\Local\Programs\Python\Python310\lib\site-packages\catboost\core.py", line 2303, in _fit
    train_params = self._prepare_train_params(
  File "c:\Users\kosmo\